This notebook calculates the impact of NHS price concessions in 2018 on organisations that are not CCGs.

In [1]:
import pandas as pd
import numpy as np
GBQ_PROJECT_ID = '620265099307'

In [2]:
## SQL query to extract additonal costs of NHS price concessions. 
## This is a modfied version of the SQL query from OpenPrescribing.net NHS price concession calculator site i.e. done at a "organisational level" 
q='''SELECT
  ncso.date AS month,
  product.bnf_code AS bnf_code,
  product.name AS product_name,
  rx.quantity AS quantity,
  rx.pct AS pct,
  ccg.org_type,
  ccg.name,
  dt.price_pence
    * rx.quantity
    * CASE WHEN
        -- For some presentations "quantity" means "number of packs" rather
        -- than e.g. tablets. In these cases we don't want to divide by the
        -- quantity value of a pack. This is implemented via a flag in our
        -- databse but this data isn't in BiqQuery so we just have a hardcoded
        -- list of BNF codes here
        product.bnf_code in ('0206010F0AACJCJ')
      THEN
        1
      ELSE
        1 / vmpp.qtyval
      END
    -- This is the "discount factor" which applies the National Average 7.2%
    -- discount to estimate Actual Cost from Net Ingredient Cost and also
    -- converts figures from pence to pounds
    * 0.00928
    AS tariff_cost,
  COALESCE(ncso.price_concession_pence - dt.price_pence, 0)
    * rx.quantity
    * CASE WHEN
        product.bnf_code in ('0206010F0AACJCJ')
      THEN
        1
      ELSE
        1 / vmpp.qtyval
      END
    * 0.00928
    AS additional_cost,
  ncso.date != rx.month AS is_projection
FROM
  dmd.ncsoconcession AS ncso
JOIN
  dmd.tariffprice AS dt
ON
  ncso.vmpp = dt.vmpp AND ncso.date = dt.date
JOIN
  dmd.product AS product
ON
  dt.product=product.dmdid
JOIN
  dmd.vmpp AS vmpp
ON
  vmpp.vppid=ncso.vmpp
JOIN
  hscic.presentation AS presentation
ON
  presentation.bnf_code = product.bnf_code
JOIN
  hscic.prescribing AS rx
ON
  rx.bnf_code = product.bnf_code
JOIN
  hscic.ccgs AS ccg #this joins to our CCG organisational data
ON
  rx.pct = ccg.code
AND
-- Where we have prescribing data for the corresponding month we use
-- that, otherwise we use the latest month of prescribing data to
-- produce an estimate
(
  rx.month = ncso.date
  OR
  (
    -- This should be set to the latest date for which we have prescribing
    -- data
    rx.month = TIMESTAMP('2018-12-01')
    AND
    ncso.date > rx.month
  )
)'''
    
all_ncso = pd.read_gbq(q, GBQ_PROJECT_ID, verbose = False, dialect = 'standard')

C:\Users\bmackenna\Anaconda3\lib\site-packages\pandas_gbq\gbq.py:556: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  "verbosity", FutureWarning, stacklevel=1)


#### Importing organisational for non-CCGs naming data from NHS Digital ODS Service

In [3]:
## import councils from NHS Digital ODS codes
councils = pd.read_csv(r'C:\Users\bmackenna\Documents\GitHub\jupyter-notebooks\nonCCG Concessions\Lauth.csv')
councils['pct'] = councils['pct'].astype('str')
councils.head(5)

,pct,name,national_grouping,high_level_health_geography,address line 1,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,postcode,open_date,close_date,null,organisation sub-type code
0,12,ISLE OF ANGLESEY UA,W00,Q99,COUNCIL OFFICES,NaN,NaN,LLANGEFNI,GWYNEDD,LL77 7TW,19960401,NaN,NaN,H
1,14,GWYNEDD UA,W00,Q99,SHIREHALL STREET,NaN,NaN,CAERNARFON,GWYNEDD,LL55 1SH,19960401,NaN,NaN,H
2,16,CONWY UA,W00,Q99,BODLONDEB,BANGOR ROAD,NaN,CONWY,GWYNEDD,LL32 8DU,19960401,NaN,NaN,H
3,18,DENBIGHSHIRE UA,W00,Q99,COUNCIL OFFICES,WYNNSTAY ROAD,NaN,RUTHIN,CLWYD,LL15 1YN,19960401,NaN,NaN,H
4,20,FLINTSHIRE UA,W00,Q99,COUNTY HALL,NaN,NaN,MOLD,CLWYD,CH7 6NB,19960401,NaN,NaN,H


In [4]:
## import "independent providers" from NHS Digital ODS codes
independent_providers = pd.read_csv(r'C:\Users\bmackenna\Documents\GitHub\jupyter-notebooks\nonCCG Concessions\ephp.csv')
independent_providers.head(5)

,pct,name,national_grouping,high_level_health_geography,address line 1,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,postcode,open_date,close_date
0,AA4,INTRAHEALTH LTD,Y54,Q74,"1ST FLOOR, WILLIAM BROWN CENTRE",MANOR WAY,NaN,PETERLEE,COUNTY DURHAM,SR8 5TW,20130401,NaN
1,AA5,COMPASS WELLBEING COMMUNITY INTEREST COMPANY,Y56,Q71,STEELS LANE HEALTH CENTRE,384-388 COMMERCIAL ROAD,NaN,LONDON,GREATER LONDON,E1 0LR,20130401,NaN
2,AA6,ASSISTED CONCEPTION UNIT LTD,Y56,Q71,LEYTONSTONE HOUSE,LEYTONSTONE,NaN,LONDON,GREATER LONDON,E11 1GA,20130401,NaN
3,AA7,LEYLAND PHYSIOTHERAPY,Y54,Q84,83 BOW LANE,NaN,NaN,LEYLAND,LANCASHIRE,PR25 4YB,20130401,NaN
4,AA8,SMART CJS,Y59,Q87,BUILDING B,KIRTLINGTON BUSINESS CENTRE,"SLADE FARM, KIRTLINGTON",KIDLINGTON,OXFORDSHIRE,OX5 3JA,20130401,NaN


In [5]:
## import hospitals from NHS Digital ODS codes
hospital_providers = pd.read_csv(r'C:\Users\bmackenna\Documents\GitHub\jupyter-notebooks\nonCCG Concessions\etr.csv')
hospital_providers.head(5)

,pct,name,national_grouping,high_level_health_geography,address line 1,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,postcode,open_date
0,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,Y54,Q83,COBBETT HOUSE,OXFORD ROAD,NaN,MANCHESTER,GREATER MANCHESTER,M13 9WL,20171001
1,R1A,WORCESTERSHIRE HEALTH AND CARE NHS TRUST,Y55,Q77,ISAAC MADDOX HOUSE,SHRUB HILL INDUSTRIAL ESTATE,NaN,WORCESTER,WORCESTERSHIRE,WR4 9RW,20110701
2,R1C,SOLENT NHS TRUST,Y59,Q87,SOLENT NHS TRUST HEADQUARTERS,HIGHPOINT VENUE,BURSLEDON ROAD,SOUTHAMPTON,HAMPSHIRE,SO19 8BR,20110401
3,R1D,SHROPSHIRE COMMUNITY HEALTH NHS TRUST,Y55,Q76,WILLIAM FARR HOUSE,MYTTON OAK ROAD,NaN,SHREWSBURY,SHROPSHIRE,SY3 8XL,20110701
4,R1E,STAFFORDSHIRE AND STOKE ON TRENT PARTNERSHIP N...,Y55,Q76,"2ND FLOOR, MORSTON HOUSE",NaN,NaN,NEWCASTLE-UNDER-LYME,STAFFORDSHIRE,ST5 1QG,20110901


In [6]:
## Selecting only nonCCG organisations to work with
nonccg_ncso  = all_ncso.loc[all_ncso.org_type !='CCG']
nonccg_ncso.head()

,month,bnf_code,product_name,quantity,pct,org_type,name,tariff_cost,additional_cost,is_projection
98,2018-04-01,0402010ADAAAAAA,Aripiprazole 10mg tablets,129,RTV,Unknown,None,8.508103,4.531954,False
225,2018-04-01,0402010ADAAADAD,Aripiprazole 5mg tablets,266,RYG,Unknown,None,20.893920,4.231680,False
254,2018-04-01,0410030A0AAADAD,Buprenorphine 2mg sublingual tablets sugar free,226,218,Unknown,None,27.863863,12.583680,False
696,2018-08-01,1304000Y0AAAAAA,Mometasone 0.1% cream,90,NCN,Unknown,None,4.952736,1.645344,False
697,2018-08-01,1304000Y0AAAAAA,Mometasone 0.1% cream,90,NDL,Unknown,None,4.955520,3.034560,False


In [7]:
## ensuring the format is consistent for pounds and pence
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [8]:
## Restricting data to 2018 calendar year
nonccg_ncso_thisyear = nonccg_ncso.loc[(nonccg_ncso["month"]>="2018-01-01") & (nonccg_ncso["month"]<="2018-12-01")]
nonccg_ncso_thisyear.head()

,month,bnf_code,product_name,quantity,pct,org_type,name,tariff_cost,additional_cost,is_projection
98,2018-04-01,0402010ADAAAAAA,Aripiprazole 10mg tablets,129,RTV,Unknown,None,8.51,4.53,False
225,2018-04-01,0402010ADAAADAD,Aripiprazole 5mg tablets,266,RYG,Unknown,None,20.89,4.23,False
254,2018-04-01,0410030A0AAADAD,Buprenorphine 2mg sublingual tablets sugar free,226,218,Unknown,None,27.86,12.58,False
696,2018-08-01,1304000Y0AAAAAA,Mometasone 0.1% cream,90,NCN,Unknown,None,4.95,1.65,False
697,2018-08-01,1304000Y0AAAAAA,Mometasone 0.1% cream,90,NDL,Unknown,None,4.96,3.03,False


### 2018 Impact - all months, all orgs, all meds

In [10]:
## Total cost impact of price consessions this year to nonCCG organisations. This will return an "unknown name" but is all nonCCGs
nonccg_ncso_thisyear.groupby('org_type').sum().sort_values(by = 'additional_cost', ascending = False)

,quantity,tariff_cost,additional_cost,is_projection
org_type,,,,
Unknown,9250711,1276379.41,5627890.92,False


In [11]:
## nonccg organisation spend grouped by indivual preparations
nonccg_ncso_thisyear.groupby('product_name').sum().sort_values(by = 'additional_cost', ascending = False)

,quantity,tariff_cost,additional_cost,is_projection
product_name,,,,
Buprenorphine 8mg sublingual tablets sugar free,2662817,682062.04,3646366.48,False
Buprenorphine 2mg sublingual tablets sugar free,3450506,414783.01,1823930.78,False
Phenoxymethylpenicillin 125mg/5ml oral solution,561600,17613.44,18634.74,False
Phenoxymethylpenicillin 250mg/5ml oral solution,331900,13230.31,9179.18,False
Naproxen 500mg tablets,48466,1989.28,7665.94,False
Diamorphine 30mg powder for solution for injection ampoules,7637,16243.72,7172.18,False
Metronidazole 400mg tablets,43417,4715.31,6723.62,False
Risperidone 500microgram tablets,69363,1931.19,6346.26,False
Diamorphine 10mg powder for solution for injection ampoules,7622,16571.03,6285.61,False


In [12]:
## monthly impact of price concessions for nonCCG organisations
nonccg_ncso_thisyear.groupby('month').sum()

,quantity,tariff_cost,additional_cost,is_projection
month,,,,
2018-01-01,222752,14003.23,12341.81,False
2018-02-01,527126,21353.33,20029.98,False
2018-03-01,654052,24725.51,23698.88,False
2018-04-01,988802,126949.47,44418.85,False
2018-05-01,972980,133551.95,870681.17,False
2018-06-01,795898,122896.73,564727.31,False
2018-07-01,830210,121639.71,626966.71,False
2018-08-01,820650,140137.93,480764.05,False
2018-09-01,764163,131730.37,738909.13,False


### Impact on individual organisations

In [13]:
## assess impact on each organisation "PCT" i.e. nonCCG organisations
nonccg_ncso_orgs = nonccg_ncso_thisyear.groupby('pct').sum().sort_values(by = 'additional_cost', ascending = False)

In [14]:
## adding in council names from organisational data name
add_councils = nonccg_ncso_orgs.reset_index().merge(councils[['pct','name']],  how = "outer", on = 'pct')

In [15]:
## adding in independent provider names
add_indprov = add_councils.merge(independent_providers[['pct','name']],  how="outer", on='pct', suffixes=("_council","_independent_provider"))

In [20]:
## finaly adding hospital name
all_nonccg_orgs = add_indprov.merge(hospital_providers[['pct','name']],  how="outer", on='pct')

In [21]:
## creating new single column with  organisation name so no blanks
all_nonccg_orgs['organisation_name'] = all_nonccg_orgs['name_council'].fillna('') + all_nonccg_orgs['name_independent_provider'].fillna('') + all_nonccg_orgs['name'].fillna('')
all_nonccg_orgs.head(5)

,pct,quantity,tariff_cost,additional_cost,is_projection,name_council,name_independent_provider,name,organisation_name
0,NMS,2321084.00,420902.13,2091871.32,False,NaN,"CHANGE, GROW, LIVE",NaN,"CHANGE, GROW, LIVE"
1,NKI,652671.00,128192.32,582024.50,False,NaN,TURNING POINT,NaN,TURNING POINT
2,NI3,278206.00,47257.92,226144.28,False,NaN,ADDACTION,NaN,ADDACTION
3,212,201394.00,36833.46,181742.50,False,LEEDS CITY COUNCIL,NaN,NaN,LEEDS CITY COUNCIL
4,RX4,197914.00,35077.43,174186.77,False,NaN,NaN,"NORTHUMBERLAND, TYNE AND WEAR NHS FOUNDATION T...","NORTHUMBERLAND, TYNE AND WEAR NHS FOUNDATION T..."


In [22]:
price_concession_impact_nonCCG = all_nonccg_orgs[['pct', 'organisation_name', 'tariff_cost', 'additional_cost']]
price_concession_impact_nonCCG

,pct,organisation_name,tariff_cost,additional_cost
0,NMS,"CHANGE, GROW, LIVE",420902.13,2091871.32
1,NKI,TURNING POINT,128192.32,582024.50
2,NI3,ADDACTION,47257.92,226144.28
3,212,LEEDS CITY COUNCIL,36833.46,181742.50
4,RX4,"NORTHUMBERLAND, TYNE AND WEAR NHS FOUNDATION T...",35077.43,174186.77
5,112,MIDDLESBROUGH COUNCIL,34264.28,164438.04
6,107,NEWCASTLE-UPON-TYNE CITY COUNCIL,24863.42,124219.57
7,NL1,SPECTRUM COMMUNITY HEALTH - CIC,24673.03,122254.48
8,DG3,THE FORWARD TRUST,23410.78,116849.06
9,114,STOCKTON-ON-TEES BOROUGH COUNCIL,22778.26,113859.05
